In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score


Using TensorFlow backend.


In [2]:
class fp_pre:
    def __init__(self,fn_weather, fn_price):
        self.r_data = pd.read_csv(fn_weather, encoding="utf-8")
        self.lc_d = pd.read_csv("../lunar_celeb_date.csv", encoding="utf-8")
        self.df_pbw = pd.read_csv(fn_price, encoding="utf-8")
        self.d_data = self.r_data["Date"]
    
    
    def prep_rdata(self):
        r_data = self.r_data
        col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
        for col in col_fix:
            old = r_data[col]
            new = []
            for i in range(len(old)):
                try:
                    new.append(float(old[i]))
                    tmp = float(old[i])
                except ValueError:
                    new.append(tmp)
            r_data[col] = new
        d_tmp = r_data['T.Max'] - r_data['T.Min']
        r_data["d_tmp"]= d_tmp
        drop_c =["Date",'T.Max', 'T.Min']
        r_data = r_data.drop(columns=drop_c)
        
        def to_zscore(df):
            col_x = df.columns.to_list()
            mean = df.mean(axis=0)
            std = df.std(axis=0)
            for i in range(len(col_x)):
                df[col_x[i]]=(df[col_x[i]]-mean[i])/std[i]
            return  df , (mean, std)
        r_data, self.recordz = to_zscore(r_data)
        self.r_data = r_data
    
    
    def join_lunar(self):
        self.r_data["Date"] = self.d_data
        self.r_data = self.r_data.join(self.lc_d.set_index("date"), on="Date")
        self.r_data = self.r_data.drop(columns="Date")
    
    
    def shift_rdata(self, shift1, shift2):
        dfs =[]
        arr_all =[]
        if shift2==0:
            df_s = self.r_data.copy()
        else:
            df_s = self.r_data.copy()
            df_s = df_s.shift(periods=shift2)
        arr_all = np.array(df_s)

        for i in range(1,shift1):
            tp = np.array(df_s.shift(periods=i))
            arr_all = np.concatenate((arr_all, tp), axis=1)
        df_all = pd.DataFrame(arr_all)
        df_all["date"] = self.d_data
        df_all = df_all.dropna()
        df_all = df_all.reset_index()
        df_all = df_all.drop(columns="index")
        self.d_data = df_all["date"]
        self.r_data = df_all
    
    
    def cutyw_rdata(self):
        def trans_to_y_w(d_date):
            year = int(d_date.split("-")[0])
            d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
            d_w = 1+ (d_day.days // 7)
            if d_w ==53:
                d_w = 52
            result =  str(year) + "_" + str(d_w)

            if d_day.days % 7 ==6:
                return result
            else:
                return np.nan
        y_w = []
        for i in range(len(self.d_data)):
            y_w.append(trans_to_y_w(self.d_data[i]))
        self.r_data["y_w"] = y_w
        self.r_data = self.r_data.dropna()
        self.r_data = self.r_data.reset_index()
        self.r_data = self.r_data.drop(columns="index")
    
    
    def join_pbw_r_data(self):
#         df_join = self.df_pbw.join(self.r_data.set_index("y_w"), on="y_w")
        df_join = self.df_pbw.join(self.r_data.set_index("y_w"), on="y_w")
        df_join = df_join.dropna()
        df_join = df_join.reset_index()
        df_join = df_join.drop(columns="index")
        self.r_data = df_join
    
    
    def dummy_market(self,d_t_dummy):
        tmp_dum = pd.DataFrame()
        for col in d_t_dummy:
            tmp_dum[col] = self.r_data[col]
            dummy = pd.get_dummies(tmp_dum[col])
            self.r_data = pd.concat([self.r_data, dummy], axis=1)
        self.r_data = self.r_data.drop(columns=d_t_dummy)
        self.x = self.r_data
#         [,"year", "week","w_avg", "w_sale", "date", "y_w", "price_diff"]
    
    
    def drop_x_data(self,drop_c):
        self.x = self.x.drop(columns=drop_c)
        self.input_units = self.x.shape[1]
    
    
    def all_prep(self, shift1, shift2,d_t_dummy):
        self.prep_rdata()
        self.join_lunar()
        self.shift_rdata(shift1, shift2)
        self.shift1 = shift1
        self.shift2 = shift2
        self.cutyw_rdata()
        self.join_pbw_r_data()
        self.dummy_market(d_t_dummy)
        self.extract_xy()
    
    
    def extract_xy(self):
#     y_date = df_join["date"]
        self.y_yw = self.r_data["y_w"]
        self.y_raw = np.array(self.r_data["price_diff"])
        
    
    
    def y_to_class(self, list_y_class=[-0.1,0,0.1]):
        y_raw = self.y_raw
        self.y_list_class = list_y_class
        y_class = []
        for i in range(len(y_raw)):
            c = 0
            y = y_raw[i]
            for j in range(len(list_y_class)):
                if y > list_y_class[j]:
                    c = c +1
            y_class.append(c)
        self.y_class = np.array(y_class)
        self.y_freq = (np.bincount(y_class), list_y_class)
        self.output_class =len(np.bincount(y_class))
    
    
    def split_data(self, year):
        y_w = self.y_yw
        y_year = []
        for  i in range(len(y_w)):
            y_year.append(int(y_w[i].split("_")[0]))
        y_year = np.array(y_year)
        self.split_info = np.unique(np.array(y_year), return_counts=True)
        b_train = y_year[:]<=year
        b_test = y_year[:]>year
        x = np.array(self.x)
        y = np.array(self.y_class)
        self.split_xy = (x[b_train,:], y[b_train] , x[b_test,:], y[b_test])
    
    def model_build(self, dinp, epochs):
        x_train, y_train, x_test, y_test = self.split_xy
        model = Sequential()
        layer_0 = Dense(units =self.input_units//dinp, input_dim = self.input_units,
                        kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_0)
        model.add(Dropout(0.25))
        layer_1 = Dense(units =self.input_units//(dinp*10),
                        kernel_initializer = "random_normal", activation = "relu")
        model.add(layer_1)
        model.add(Dropout(0.25))
#         layer_1 = Dense(units =(self.shift1+1),kernel_initializer = "random_normal", activation = "relu")
#         model.add(layer_1)
#         model.add(Dropout(0.25))
        layer_out = Dense(units = self.output_class,kernel_initializer = "random_normal", 
                          activation = "softmax")
        model.add(layer_out)
        #     model.summary()
        model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])
        self.train_model = model

    def model_test(self):
        x_train, y_train, x_test, y_test = self.split_xy
        train_history = self.train_model.fit(x = x_train, y = np_utils.to_categorical(y_train), 
                                  validation_split = 0.1, epochs =epochs, verbose = 0)
        pre = self.train_model.predict_classes(x_test)
        acc = round(accuracy_score(y_test, pre)*100,2)
        return acc

In [3]:
w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/flower_price_byweek/*")
select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
# 修改 select_t 可以拿不同測站 vs花花朵
st, flower = select_t[0]
fn_weather = w_list[st]
fn_price = p_list[flower]
print(fn_weather, fn_price)


../result/merge_weather\C0F9L0_后里.csv ../result/flower_price_byweek\Anthurium_pbyweek.csv


In [12]:
fp = fp_pre(fn_weather, fn_price)


,Date,Temperature,T.Max,T.Min,Precp,RH,StnPres,WS,WSGust
1846,2019-03-05,19.5,25.3,15.1,7.0,83,984.1,1.5,8.4
1847,2019-03-06,18.7,24.6,15.9,24.0,91,985.0,1.9,9.2
1848,2019-03-07,13.1,17.0,11.5,33.0,99,986.7,2.4,9.1
1849,2019-03-08,14.2,17.6,11.3,15.0,98,986.8,1.5,5.9
1850,2019-03-09,18.3,22.1,15.5,71.0,97,982.7,1.1,6.0
1851,2019-03-10,18.3,22.0,15.8,23.5,91,983.4,1.6,11.0
1852,2019-03-11,16.9,18.8,14.4,1.0,88,984.1,1.3,6.4
1853,2019-03-12,19.3,24.9,13.6,0.0,71,986.0,1.4,7.5
1854,2019-03-13,20.9,25.1,17.1,0.0,67,988.7,0.9,6.5
1855,2019-03-14,19.5,25.3,16.3,0.0,81,989.5,1.8,8.7


In [5]:
# bullid class 讀取檔案
fp = fp_pre(fn_weather, fn_price)
# 花價格 + 天氣 + 節日後刪除資料
drop_c = ["year","w_avg", "w_sale", "date", "y_w", "price_diff"]
# 目前轉成 one-hot encoding 的資料
d_t_dummy = ["market", "week"]
# 設定數字 (總共要取幾天的資料,挑過幾天( 8 則跳兩週)
fp.all_prep(28,1,d_t_dummy)
fp.drop_x_data(drop_c)
# 設定結果 y pct_change 切分點 [-0.1 ,0, 0.1] 則切成 [-0.1以下, -0.1 ~ 0,0 ~ 0.1, 0.1 以上]
fp.y_to_class([-0.1,0,0.1])
fp.split_data(2017)
fp.d_data

0       2014-01-29
1       2014-01-30
2       2014-01-31
3       2014-02-01
4       2014-02-02
5       2014-02-03
6       2014-02-04
7       2014-02-05
8       2014-02-06
9       2014-02-07
10      2014-02-08
11      2014-02-09
12      2014-02-10
13      2014-02-11
14      2014-02-12
15      2014-02-13
16      2014-02-14
17      2014-02-15
18      2014-02-16
19      2014-02-17
20      2014-02-18
21      2014-02-19
22      2014-02-20
23      2014-02-21
24      2014-02-22
25      2014-02-23
26      2014-02-24
27      2014-02-25
28      2014-02-26
29      2014-02-27
           ...    
1728    2018-12-01
1729    2018-12-02
1730    2018-12-03
1731    2018-12-04
1732    2018-12-05
1733    2018-12-08
1734    2018-12-09
1735    2018-12-10
1736    2018-12-11
1737    2018-12-12
1738    2018-12-13
1739    2018-12-14
1740    2018-12-15
1741    2018-12-16
1742    2018-12-17
1743    2018-12-18
1744    2018-12-19
1745    2018-12-20
1746    2018-12-21
1747    2018-12-22
1748    2018-12-23
1749    2018

In [19]:
for dinp in range(2,10):
    print(dinp, end="\t")
    fp.model_build(dinp, 5)
    for run_times in range(5):
#         print(epochs, end="\t")
        print(fp.model_test(), end="\t")
    print()

2	29.02	29.8	32.94	29.8	33.73	
3	32.16	31.76	32.55	30.98	30.59	
4	32.94	30.2	29.41	30.98	30.59	
5	32.55	30.59	30.98	28.63	28.24	
6	30.59	33.73	32.16	30.59	32.16	
7	31.37	30.98	32.94	29.8	27.06	
8	27.45	30.2	29.8	31.76	28.24	
9	34.51	31.37	32.94	31.37	32.16	
